# Retrieve heating system info per building based on SiL data

In [2]:
import pandas as pd
import geopandas as gpd

In [2]:
shared_folder = '/Users/catarina/switchdrive/IPESE-DESL_data_exchange/SDEWES2025/data/'
Qbuildings_file = shared_folder + 'buildings_SDEWES.gpkg'
Qbuildings_exp_file = 'QBuildings_SDEWES_exp.csv'
SiL_file = '/Users/catarina/switchdrive/SIL_UrbanTwin_data_exchange/Data/Gas and CAD network/Point_Energie.geojson'
SiL_cleaned_file = 'SiL_cleaned.csv'


In [3]:
def read_file(file):
    """
    Function to read a file and return a dataframe
    
    Parameters:
    - file: str ; Path to the file
    
    Returns:
    - df: dataframe ; Dataframe with the file content
    """
    # get extension of file
    file_ext = file.split('.')[-1]
    # check if the file is a .gpkg, .geojson or .csv. If none, raise an error
    if file_ext in ['gpkg', 'geojson']:
        df = gpd.read_file(file)
    elif file_ext == 'csv':
        df = pd.read_csv(file)
    else:
        raise ValueError('File must be a .gpkg, .geojson or .csv file')
    return df

In [4]:
def explode_qbuildings(qbuildings):
    # read the QBuildings file
    qbuildings_df = read_file(qbuildings)

    # Split the egid, ratio and in_class columns into separate columns based on /
    for column in ['egid','ratio','id_class']:
        qbuildings_df[column] = qbuildings_df[column].str.split('/')

    # Create a copy of the egid column
    qbuildings_df['egid_new'] = qbuildings_df['egid']

    # Explode the columns
    qbuildings_df = qbuildings_df.explode('egid')

    # apply lambda function to extract the correct ratio and id_class values
    for column in ['ratio','id_class']:
        qbuildings_df[column] = qbuildings_df.apply(lambda row: row[column][row['egid_new'].index(str(row['egid']))], axis=1)

    # drop the egid_new column
    qbuildings_df.drop(columns='egid_new', inplace=True)
    # Convert ration column to float
    qbuildings_df['ratio'] = qbuildings_df['ratio'].astype(float)

    # calculate new values of building parameters, based on the ratio TO DO: check if solar_gain_factor_signature_m2 should also be multiplied by the ratio
    for column in ['area_era_m2','energy_heating_signature_kWh_y','energy_cooling_signature_kWh_y','power_heating_signature_kW','power_cooling_signature_kW','total_annual_irr_kWh_y','roof_annual_irr_kWh_y','facade_annual_irr_kWh_y','area_solar_m2','energy_hotwater_signature_kWh_y','power_hotwater_signature_W','waste_signature_kg_y','waste_signature_kW','area_net_floor_m2','area_footprint_m2','area_facade_m2','area_facade_solar_m2','area_roof_solar_m2','capita_cap','energy_el_kWh_y','solar_gain_factor_signature_m2','area_roof_m2','energy_heating_2024_kWh_y','energy_hotwater_2024_kWh_y']:
        qbuildings_df[column] = qbuildings_df[column] * qbuildings_df['ratio']

    # create a new column called id_building new, check if the id_building is a duplicate, and if so, add _1, _2, etc to the id_building
    qbuildings_df['id_building_new'] = qbuildings_df['id_building']
    qbuildings_df['id_building_new'] = qbuildings_df.groupby('id_building_new').cumcount()
    qbuildings_df['id_building'] = qbuildings_df.apply(lambda row: row['id_building'] if row['id_building_new'] == 0 else f"{row['id_building']}_{row['id_building_new']+1}", axis=1)
    # drop the id_building_new column
    qbuildings_df.drop(columns='id_building_new', inplace=True)

    # remove row with ratio = 0
    qbuildings_df = qbuildings_df[qbuildings_df['ratio'] != 0]

    qbuildings_df['ratio'] = 1.0 # ratio must be float

    return qbuildings_df

Qbuildings_exploded = explode_qbuildings(Qbuildings_file)
Qbuildings_exploded.to_csv('QBuildings_SDEWES_exp.csv',index=False)


In [5]:
def clean_SiL(SiL_file):
    """
    Function to remove qlEner_iEGID duplicates from the SiL dataframe keeping the row with the vaule with the highest qlEner_Densité

    Parameters:
    - SiL_file: str ; path to the SiL file. Can be a .gpkg, .geojson or .csv file

    Returns
    - updated SiL dataframe
    """

    # Read SiL file
    SiL_df = read_file(SiL_file)

    # Sort the dataframe by qlEner_Densité
    SiL_df = SiL_df.sort_values('qlEner_Densité', ascending=False)

    # Drop duplicates based on qlEner_iEGID
    cleaned_SiL_df = SiL_df.drop_duplicates(subset='qlEner_iEGID', keep='first')


    return cleaned_SiL_df

#clean_SiL=clean_SiL(SiL_file)
# clean_SiL.to_csv('SiL_cleaned.csv',index=False)

In [6]:
def update_QBuildings(qbuildings,update_data,QB_criteria,data_criteria,columns,data_new_name=None):
    """
    Function to receive a path for a QBuildings file, a file path with data to be added to the QBuildings file, a list of columns with the data to be added and a list of criteria to match the data.
    The function will update the QBuildings file with the data provided in the dataframe, based on the criteria.

    Parameters:
    - qbuildings: str ; Path to the QBuildings file. Can be a .gpkg, a .geojson or .csv file
    - update_data: str ; Data to be added to the QBuildings file
    - columns: list ; Columns with the data to be added
    - QB_criteria: str ; Column in QBuildings to match the data
    - data_criteria: str ; Column in data to update the QBuildings data
    - data_new_name: list ; New name of the data to be added. Default is None

    Returns:
    - Updated QBuildings dataframe
    """

    # read the QBuildings file
    qbuildings_df = read_file(qbuildings)
    # read the data to be added
    update_data_df = read_file(update_data)

    # Create columns with the data to be added and add the data to the QBuildings dataframe based on the criteria.
    for column in columns:
        qbuildings_df[column] = None
        for index, row in update_data_df.iterrows():
            qbuildings_df.loc[qbuildings_df[QB_criteria] == row[data_criteria], column] = row[column]

    # Rename the columns with the new data
    if data_new_name:
        qbuildings_df.rename(columns=dict(zip(columns,data_new_name)), inplace=True)

    return qbuildings_df

# Update QBuildings

In [7]:
qbuildings = Qbuildings_exp_file
update_data = SiL_cleaned_file
QB_criteria = 'egid'
data_criteria = 'qlEner_iEGID'
columns = ['qlEner_sType','qlEner_sRecommendation']
data_new_name = ['heating_system_2024','recommendation']

Qbuildings_updated = update_QBuildings(qbuildings,update_data,QB_criteria,data_criteria,columns,data_new_name)

heat_source_dic = {'Gaz': 'Gas', 'Mazout': 'Oil', 'CAD': 'DHN'}
recommendation_dic = {'Sondes': 'HeatPump_Geothermal', 'CAD actuel': 'DHN', 'CAD futur': 'DHN_futur', 'CAD prévu': 'DHN_2050', 'Déjà raccordé': '2024', 'PAC Air Eau': 'HeatPump_Air', 'Sondes si rénovation': 'HeatPump_Geothermal', 'Gaz conservé': 'Gas', 'PAC si rénovation': 'HeatPump_Air', 'Gaz remplace mazout': 'Gas', 'Déjà raccordé remplaçable par sonde': '2024'}
# Update QBuildings with heating system info and recommendation
Qbuildings_updated['heating_system_2024']=Qbuildings_updated['heating_system_2024'].map(heat_source_dic)
Qbuildings_updated['recommendation']=Qbuildings_updated['recommendation'].map(recommendation_dic)
# fill none values in heating_system_2024 with the source_heating column
Qbuildings_updated['heating_system_2024'] = Qbuildings_updated['heating_system_2024'].fillna(Qbuildings_updated['source_heating'])

# In column id_class, replace XIII by V
Qbuildings_updated['id_class'] = Qbuildings_updated['id_class'].replace('XIII','V')

Qbuildings_updated.to_csv('../../QBuildings/QBuildings_SDEWES_updated.csv',index=False)

In [5]:
# read the QBuildings file
Qbuildings_file = '../../QBuildings/QBuildings_SDEWES_updated.csv'
qbuildings_df = read_file(Qbuildings_file)

# keep rows where in column 'heating_system_2024' is equal to Gas or Oil
qbuildings_df = qbuildings_df[qbuildings_df['heating_system_2024'].isin(['Gas','Oil','DHN'])]
qbuildings_df.to_csv('../../QBuildings/QBuildings_SDEWES_updated_2.csv',index=False)

In [62]:
from reho.model.reho import *
reader = QBuildingsReader()
qbuildings_data = reader.read_csv(buildings_filename='/Users/catarina/Documents/REHO/scripts/Pathways/QBuildings/QBuildings_SDEWES_updated.csv',nb_buildings=2)

In [66]:
for key in qbuildings_data['buildings_data'].keys():
    print(qbuildings_data['buildings_data'][key].keys())

dict_keys(['geometry', 'id_building', 'egid', 'id_class', 'class', 'ratio', 'status', 'period', 'n_p', 'ERA', 'SolarRoofArea', 'area_facade_m2', 'height_m', 'count_floor', 'source_heating', 'source_hotwater', 'U_h', 'HeatCapacity', 'T_comfort_min_0', 'Th_supply_0', 'Th_return_0', 'Tc_supply_0', 'Tc_return_0', 'x', 'y', 'z', 'transformer', 'energy_heating_signature_kWh_y', 'energy_cooling_signature_kWh_y', 'energy_hotwater_signature_kWh_y', 'energy_el_kWh_y', 'roof_annual_irr_kWh_y', 'facade_annual_irr_kWh_y'])
dict_keys(['geometry', 'id_building', 'egid', 'id_class', 'class', 'ratio', 'status', 'period', 'n_p', 'ERA', 'SolarRoofArea', 'area_facade_m2', 'height_m', 'count_floor', 'source_heating', 'source_hotwater', 'U_h', 'HeatCapacity', 'T_comfort_min_0', 'Th_supply_0', 'Th_return_0', 'Tc_supply_0', 'Tc_return_0', 'x', 'y', 'z', 'transformer', 'energy_heating_signature_kWh_y', 'energy_cooling_signature_kWh_y', 'energy_hotwater_signature_kWh_y', 'energy_el_kWh_y', 'roof_annual_irr_kWh_

# Pathway file

In [26]:
file = ('/Users/catarina/Documents/REHO/scripts/Pathways/QBuildings/Pathway_SDEWES_updated.csv')

pathways_df = pd.read_csv(file)

# keep only egid and heating_system_2024 columns
pathways_df = pathways_df[['egid','heating_system_2024','recommendation']]

# substitute value 2024 in column recommendation by the value in heating_system_2024
pathways_df['heating_system_2050'] = pathways_df.apply(lambda row: row['heating_system_2024'] if row['recommendation'] == '2024' else row['recommendation'], axis=1)

pathways_df['heating_system_2050'] = pathways_df.apply(lambda row: 'DHN' if row['heating_system_2050'] in ['DHN_futur', 'DHN_2050'] else row['heating_system_2050'], axis=1)

pathways_df['heating_system_2024'] = pathways_df['heating_system_2024'].replace('DHN','DHN_hex')
pathways_df['heating_system_2024'] = pathways_df['heating_system_2024'].replace('District heat','DHN_hex')
pathways_df['heating_system_2024'] = pathways_df['heating_system_2024'].replace('District heat/Wood','DHN_hex')
pathways_df['heating_system_2024'] = pathways_df['heating_system_2024'].replace('Other/District heat','DHN_hex')
pathways_df['heating_system_2024'] = pathways_df['heating_system_2024'].replace('Other/District heat/Gas','DHN_hex')
pathways_df['heating_system_2024'] = pathways_df['heating_system_2024'].replace('District heat/Other','DHN_hex')

pathways_df['heating_system_2024'] = pathways_df['heating_system_2024'].replace('Gas','NG_Boiler')
pathways_df['heating_system_2024'] = pathways_df['heating_system_2024'].replace('Other','NG_Boiler')
pathways_df['heating_system_2024'] = pathways_df['heating_system_2024'].replace('Gas/Oil','NG_Boiler')
pathways_df['heating_system_2024'] = pathways_df['heating_system_2024'].replace('Not determined','NG_Boiler')
pathways_df['heating_system_2024'] = pathways_df['heating_system_2024'].replace('Gas/Other','NG_Boiler')
pathways_df['heating_system_2024'] = pathways_df['heating_system_2024'].replace('Gas/Other/Oil','NG_Boiler')
pathways_df['heating_system_2024'] = pathways_df['heating_system_2024'].replace('Gas/Other/Electricity/Oil/District heat','NG_Boiler')
pathways_df['heating_system_2024'] = pathways_df['heating_system_2024'].replace('Other/Gas','NG_Boiler')
pathways_df['heating_system_2024'] = pathways_df['heating_system_2024'].replace('None/Gas','NG_Boiler')
pathways_df['heating_system_2024'] = pathways_df['heating_system_2024'].replace('None','NG_Boiler')
pathways_df['heating_system_2024'] = pathways_df['heating_system_2024'].replace('Gas/Oil/Electricity/Other','NG_Boiler')
pathways_df['heating_system_2024'] = pathways_df['heating_system_2024'].replace('Gas/Oil/Other/District heat','NG_Boiler')
pathways_df['heating_system_2024'] = pathways_df['heating_system_2024'].replace('Gas/Wood/Oil','NG_Boiler')
pathways_df['heating_system_2024'] = pathways_df['heating_system_2024'].replace('Gas/Oil/Other','NG_Boiler')
pathways_df['heating_system_2024'] = pathways_df['heating_system_2024'].replace('Gas/Electricity','NG_Boiler')
pathways_df['heating_system_2024'] = pathways_df['heating_system_2024'].replace('Gas/Other/Oil/District heat','NG_Boiler')
pathways_df['heating_system_2024'] = pathways_df['heating_system_2024'].replace('Gas/Other/Oil/Electricity','NG_Boiler')
pathways_df['heating_system_2024'] = pathways_df['heating_system_2024'].replace('Gas/District heat','NG_Boiler')
pathways_df['heating_system_2024'] = pathways_df['heating_system_2024'].replace('Gas/Other/District heat','NG_Boiler')

pathways_df['heating_system_2024'] = pathways_df['heating_system_2024'].replace('Oil','OIL_Boiler')
pathways_df['heating_system_2024'] = pathways_df['heating_system_2024'].replace('Other/Oil/Gas','OIL_Boiler')
pathways_df['heating_system_2024'] = pathways_df['heating_system_2024'].replace('Other/Oil','OIL_Boiler')
pathways_df['heating_system_2024'] = pathways_df['heating_system_2024'].replace('Oil/Other','OIL_Boiler')
pathways_df['heating_system_2024'] = pathways_df['heating_system_2024'].replace('Oil/Gas/Other','OIL_Boiler')
pathways_df['heating_system_2024'] = pathways_df['heating_system_2024'].replace('Oil/Other/Gas','OIL_Boiler')
pathways_df['heating_system_2024'] = pathways_df['heating_system_2024'].replace('None/Oil/Other/Gas','OIL_Boiler')
pathways_df['heating_system_2024'] = pathways_df['heating_system_2024'].replace('Oil/Not determined/Other/District heat','OIL_Boiler')
pathways_df['heating_system_2024'] = pathways_df['heating_system_2024'].replace('Oil/Gas','OIL_Boiler')
pathways_df['heating_system_2024'] = pathways_df['heating_system_2024'].replace('Oil/Gas/Wood','OIL_Boiler')
pathways_df['heating_system_2024'] = pathways_df['heating_system_2024'].replace('Oil/Other/Not determined','OIL_Boiler')
pathways_df['heating_system_2024'] = pathways_df['heating_system_2024'].replace('Oil/District heat','OIL_Boiler')
pathways_df['heating_system_2024'] = pathways_df['heating_system_2024'].replace('Oil/Other/District heat','OIL_Boiler')
pathways_df['heating_system_2024'] = pathways_df['heating_system_2024'].replace('None/Oil/Gas','OIL_Boiler')

pathways_df['heating_system_2024'] = pathways_df['heating_system_2024'].replace('Air','HeatPump_Air')
pathways_df['heating_system_2024'] = pathways_df['heating_system_2024'].replace('Other/Air','HeatPump_Air')
pathways_df['heating_system_2024'] = pathways_df['heating_system_2024'].replace('Geothermal probe','HeatPump_Geothermal')

pathways_df['heating_system_2024'] = pathways_df['heating_system_2024'].replace('Electricity','ElectricalHeater')
pathways_df['heating_system_2024'] = pathways_df['heating_system_2024'].replace('Wood','WOOD_Stove')
pathways_df['heating_system_2024'] = pathways_df['heating_system_2024'].replace('Wood/Other','WOOD_Stove')

pathways_df['heating_system_2050'] = pathways_df['heating_system_2050'].fillna('HeatPump_Air')
pathways_df['heating_system_2050'] = pathways_df['heating_system_2050'].replace('DHN','DHN_hex')
pathways_df['heating_system_2050'] = pathways_df['heating_system_2050'].replace('Gas','NG_Boiler')
pathways_df['heating_system_2050'] = pathways_df['heating_system_2050'].replace('Oil','OIL_Boiler')

pathways_df.drop(['recommendation'], axis=1, inplace=True)

pathways_df['electric_system_2024'] = ''
pathways_df['electric_system_2050'] = 'PV'

# rename column heating_system_2024 by heating_system_2025
pathways_df.rename(columns={'heating_system_2024': 'heating_system_2025'}, inplace=True)
pathways_df.rename(columns={'electric_system_2024': 'electric_system_2025'}, inplace=True)

# print unique values in column heating_system_2024
unique_values = pathways_df['heating_system_2025'].unique()
print(f"Unique values in 'heating_system_2025' column:{unique_values}")

pathways_df.to_csv('../../QBuildings/Pathway_SDEWES_test.csv', index=False)

#print(pathways_df)

Unique values in 'heating_system_2025' column:['NG_Boiler' 'OIL_Boiler' 'DHN_hex' 'ElectricalHeater' 'WOOD_Stove'
 'HeatPump_Air' 'HeatPump_Geothermal']


## Divide Transformers with too many buildings in half

In [37]:
import numpy as np
import pandas as pd

file = ('/Users/catarina/Documents/REHO/scripts/Pathways/QBuildings/QBuildings_SDEWES_updated_no_EH.csv')

pathways_df = pd.read_csv(file)

pathways_df['transformer_new'] = pathways_df['transformer']

transf_list = pathways_df['transformer'].unique().tolist()

for transform in transf_list:
    # if number of rows > 100
    group = pathways_df[pathways_df['transformer'] == transform]

    # If group size > 200, split into 3 parts
    if len(group) > 200:
        n_rows = len(group)
        span = np.linspace(0, n_rows, 4, endpoint=True, dtype=int)

        # Assign new transformer values in three parts
        indices = group.index
        pathways_df.loc[indices[span[0]:span[1]], 'transformer_new'] = f'{transform}_2'
        pathways_df.loc[indices[span[1]:span[2]], 'transformer_new'] = f'{transform}_3'
        pathways_df.loc[indices[span[2]:span[3]], 'transformer_new'] = f'{transform}_4'

    elif len(group) > 100:
        n_rows = len(group)
        span = np.linspace(0, n_rows, 3, endpoint=True, dtype=int)

        # Assign new transformer values in three parts
        indices = group.index
        pathways_df.loc[indices[span[0]:span[1]], 'transformer_new'] = f'{transform}_2'
        pathways_df.loc[indices[span[1]:span[2]], 'transformer_new'] = f'{transform}_3'

pathways_df.to_csv('../../QBuildings/QBuildings_SDEWES_updated_no_EH_transf.csv', index=False)


KeyboardInterrupt: 

In [2]:
import numpy as np
import pandas as pd

file = ('/Users/catarina/Documents/REHO/scripts/Pathways/QBuildings/QBuildings_SDEWES_updated_no_EH.csv')

pathways_df = pd.read_csv(file)

n_rows = len(pathways_df)

n_groups = n_rows // 70

span = np.linspace(0, n_rows, n_groups, endpoint=True, dtype=int)
indices = pathways_df.index
for i in range(1, n_groups):
    pathways_df.loc[indices[span[i-1]:span[i]], 'transformer_new'] = f'group_{i}'
pathways_df.to_csv('../../QBuildings/QBuildings_SDEWES_updated_no_EH_2.csv', index=False)

In [15]:
import numpy as np
import pandas as pd

transf = pd.read_csv('../../QBuildings/QBuildings_SDEWES_updated_no_EH_2.csv', usecols=['transformer_new'])
transf_list = transf['transformer_new'].unique().tolist()


results_folder = ('/Users/catarina/switchdrive/IPESE-DESL_data_exchange/SDEWES2025/1st Results')

for transform in transf_list:
    results = pd.read_pickle(f'{results_folder}/pathway_{transform}.pickle')
    results = results['pathway']

    results_max_PV = results['PV_max']['df_Unit']
    PV_max = results_max_PV.loc[(results_max_PV.index.str.contains('PV')) & ~(results_max_PV.index.str.contains('district'))]
    PV_max = PV_max['Units_Mult'].sum()

    results_2050_PV = results[5]['df_Unit']
    PV_2050 = results_2050_PV.loc[(results_2050_PV.index.str.contains('PV')) & ~(results_2050_PV.index.str.contains('district'))]
    PV_2050 = PV_2050['Units_Mult'].sum()

    test = PV_max - PV_2050
    print(f'Transformer: {transform}, PV_max: {PV_max}, PV_2050: {PV_2050}, Difference: {test}')


Transformer: group_1, PV_max: 4464.639999999999, PV_2050: 4476.58503095255, Difference: -11.945030952550951
Transformer: group_2, PV_max: 3750.0800000000004, PV_2050: 3762.49499335108, Difference: -12.41499335107983
Transformer: group_3, PV_max: 4843.2, PV_2050: 4854.317919043891, Difference: -11.117919043891561
Transformer: group_4, PV_max: 4405.76, PV_2050: 4417.18357178876, Difference: -11.423571788759546
Transformer: group_5, PV_max: 6132.16, PV_2050: 6144.2241001169195, Difference: -12.06410011691969
Transformer: group_6, PV_max: 6673.919999999999, PV_2050: 6684.47401264571, Difference: -10.554012645710827
Transformer: group_7, PV_max: 4486.400000000001, PV_2050: 4499.00194858386, Difference: -12.601948583859667
Transformer: group_8, PV_max: 4130.5599999999995, PV_2050: 4142.10426967232, Difference: -11.544269672320297
Transformer: group_9, PV_max: 3758.72, PV_2050: 3770.81207582478, Difference: -12.092075824780295
Transformer: group_10, PV_max: 3369.9200000000005, PV_2050: 3381.4